<a href="https://colab.research.google.com/github/Jake0925/Pytorch/blob/main/%EB%8B%A8%EC%88%9C%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://wikidocs.net/55409

# 단순 선형 회귀 구현하기

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
torch.manual_seed(1)

아래 데이터는 y = 2x를 가정된 상태에서 만들어진 데이터로 우리는 이미 정답이 W=2, b=0임을 알고 있는 상태입니다.

모델이 이 두 W와 b의 값을 제대로 찾아내도록 하는 것이 목표입니다.

In [4]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]]) # x
y_train = torch.FloatTensor([[2], [4], [6]]) # y

In [5]:
# nn.Linear()는 입력의 차원, 출력의 차원을 인수로 받습니다.
# 하나의 입력에 대해서 하나의 출력을 가지므로, 입력 차원과 출력 차원 모두 1을 인수로 사용하였습니다
# 모델을 선언 및 초기화. 단순 선형 회귀이므로 input_dim=1, output_dim=1.
model = nn.Linear(1,1)

In [6]:
# model에는 가중치 W와 편향 b가 저장되어져 있습니다. 이 값은 model.parameters()라는 함수를 사용하여 불러올 수 있는데, 한 번 출력해보겠습니다.
#2개의 값이 출력되는데 첫번째 값이 W고, 두번째 값이 b에 해당됩니다
# 두 값 모두 현재는 랜덤 초기화가 되어져 있습니다.
# 그리고 두 값 모두 학습의 대상이므로 requires_grad=True가 되어져 있는 것을 볼 수 있습니다.

print(list(model.parameters()))

[Parameter containing:
tensor([[0.5153]], requires_grad=True), Parameter containing:
tensor([-0.4414], requires_grad=True)]


In [10]:
# 이제 옵티마이저를 정의합니다. model.parameters()를 사용하여 W와 b를 전달합니다.
# 학습률(learning rate)은 0.01로 정합니다.
# optimizer 설정. 경사 하강법 SGD를 사용하고 learning rate를 의미하는 lr은 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [11]:
# 전체 훈련 데이터에 대해 경사 하강법을 100회 반복
nb_epochs = 100
for epoch in range(nb_epochs+1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train) # <== 파이토치에서 제공하는 평균 제곱 오차 함수

    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward() # backward 연산
    # W와 b를 업데이트
    optimizer.step()

    if epoch % 100 == 0:
    # 100번마다 로그 출력
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch    0/100 Cost: 13.103541
Epoch  100/100 Cost: 0.002791


In [12]:
# Cost의 값이 매우 작습니다. W와 b의 값도 최적화가 되었는지 확인해봅시다.
# x에 임의의 값 4를 넣어 모델이 예측하는 y의 값을 확인해보겠습니다.

# 임의의 입력 4를 선언
new_var =  torch.FloatTensor([[4.0]])
# 입력한 값 4에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) # forward 연산
# y = 2x 이므로 입력이 4라면 y가 8에 가까운 값이 나와야 제대로 학습이 된 것
print("훈련 후 입력이 4일 때의 예측값 :", pred_y)

훈련 후 입력이 4일 때의 예측값 : tensor([[7.8943]], grad_fn=<AddmmBackward0>)


In [ ]:
# 사실 이 문제의 정답은 y=2x 가 정답이므로 y값이 8에 가까우면 W와 b의 값이 어느정도 최적화가 된 것으로 볼 수 있습니다.
# 실제로 예측된 y값은 7.8943로 8에 매우 가깝습니다.

In [14]:
# W와 b의 값을 출력해보겠습니다.
print(list(model.parameters()))

[Parameter containing:
tensor([[1.9388]], requires_grad=True), Parameter containing:
tensor([0.1391], requires_grad=True)]


W의 값이 2에 가깝고, b의 값이 0에 가까운 것을 볼 수 있습니다.

 H(x)식에 입력 x로부터 예측된 y를 얻는 것을 **forward** 연산이라고 합니다.

 학습 전, prediction = model(x_train)은 x_train으로부터 예측값을 리턴하므로 forward 연산입니다.

 학습 후, pred_y = model(new_var)는 임의의 값 new_var로부터 예측값을 리턴하므로 forward 연산입니다.

 학습 과정에서 비용 함수를 미분하여 기울기를 구하는 것을 **backward** 연산이라고 합니다

 cost.backward()는 비용 함수로부터 기울기를 구하라는 의미이며 backward 연산입니다.